# General Information

This is a data tabulating function for X,Y data plus scalars.

Variables to change are located in .getuserinputs().

Everything else is calculated from the class assuming it has the data structure outlined in XY + Scalar Data Class.

# Import Packages

In [1]:
import numpy as np
import pandas as pd
import os

import glob
from pathlib import Path
import time

import SPD_JV_Class

# XY + Scalar Data Class

A General class section has a few main functions to work with the plotter:

A) __init__(): defines variables stored in self, it has:

     - self.ID = ID of Class (e.g. JV)
     
     - self.filepath = full path to file
     - self.filefolder = full path to folder holding file
     - self.filename = name of file including extension (e.g. JVFile.csv)
     - self.classname = name of class made from file (e.g. JVFile)
     
     - self.loadedlist: all loaded variable names
             self.load (0=not analyzed, 1=analyzed)
     - self.vectorlist: all calculated vector names
              self.vetcor (0=not analyzed, 1=analyzed)
     - self.scalarlist: all calculated scalar names
              self.scalar (0=not analyzed, 1=analyzed)

     - self.Varaible: jsc --> Jsc
     - self.Name: jsc --> Short Circuit Current Density / Jsc (mA/cm²)
     - self.NameRev: opposite of Name
     - self.Label: jsc --> Jsc (mA/cm²)
     - self.Unit: jsc --> mA/cm²

B) loadfile(fullpathtofile): loads file, sets self.load = 1, calculates all values in self.loadedlist

C) createlists(): creates lists and vectors for management of class

D) calcvectors(): sets self.vector = 1, calculates all vectors in self.vectorlist

E) calcscalars(): sets self.scalar = 1, calculates all vectors in self.vectorlist

F) getparamvals(inputvallist): returns scalars for varaible inputs

# Setup XY + Scalar Data Tabulator

This class uses inputs in 'getuserinputs' to manage workup without anything specific to the class.

In [2]:
class XYTabulator:
    
    # Takes User Inputs
    def getuserinputs(self):
        
        #standard inputs to change
        self.classname = SPD_JV_Class.JVCurve
        self.loadfolder = 'C:\\Users\\seand\\Documents\\Data\\JV\\'
        self.outputfodler = 'C:\\Users\\seand\\Documents\\Data\\JV\\Analyzed\\'
        self.xindex = 0
        self.yindex = 1
        
        # idea is timeseries will dictate if the list of devices are timeseries
        self.timeseries = 0

        #these can be changed but it is not recomended
        # nothing at the moment. 
        pd.set_option('display.max_rows', 5000)
        pd.set_option('display.max_columns', 20)
        
        #chooses what timevalues to display, to see options remove comment from code below
        """
        self.dummytime = self.classname()
        self.dummytime.createlists()
        print(self.dummytime.timelist)
        """
        if self.timeseries == 0:
            self.time_metrics = ['date_time', 'time12_time']
        else:
            self.time_metrics = ['epoch_time']

    
    # builds data set
    def __init__(self):
        
# GET USER INFO AND FILE INFO
          
        #create variables from .getuserinputs()
        self.getuserinputs()
        
        # create dummy file of class, call .createlists on dummy class, pull x and y 
        self.dummy = self.classname()  
        self.dummy.createlists()
        self.xydict = {'x':self.dummy.loadedlist[self.xindex],'y':self.dummy.loadedlist[self.yindex]}
        
        # move into folder, make list of txt files in it
        os.chdir(self.loadfolder)
        self.listoffiles = glob.glob('*csv')
        
        # calculate numscalars and devices (size of each aray, # of arrays)
        self.numdevs = len(self.listoffiles)
        self.numscalars = len(self.dummy.scalarlist)
        
# CREATE DICTIONARIES
        
        # create dictionary { filename : classfilename } where class file name is same name - .txt
        # this lets us call the class via the txt name file
        self.filestoclass = {}
        for file in self.listoffiles:
            self.filestoclass[file] = file.rsplit(".", 1)[0]
        
        # make dictionary { scalarvariable : scalarvariable + "_values" } (e.g. jsc --> jsc_list) 
        # this lets us call an array to hold all scalars via the scalar name file
        self.scalartoarrayofvalues = {}
        for scalar in self.dummy.scalarlist:
            self.scalartoarrayofvalues[scalar] = scalar + '_valuelist'
            self.scalartoarrayofvalues[scalar] = np.zeros(self.numdevs)
            
        # make dictionary { timevariable : timevariable + "_values" } (e.g. time12_time --> time12_time_list) 
        # this lets us call an array to hold all time data via the time variable name
        self.timetoarrayofvalues = {}
        for timemetric in self.dummy.timelist:
            self.timetoarrayofvalues[timemetric] = timemetric + '_valuelist'
            self.timetoarrayofvalues[timemetric] = []
            
# BUILD LISTS FROM VECTORS IN CLASS            
        
        #cycles through txt files, calculates neccisary parameters, and adds to arrays
        for z, value in enumerate(self.listoffiles):            
            # call class on txt file
            self.filestoclass[str(value)] = self.classname()
            # load file
            self.filestoclass[str(value)].loadfile(self.loadfolder+value)
            # load times
            self.filestoclass[str(value)].loadtime(self.loadfolder+value)
            # calc vectors
            self.filestoclass[str(value)].calcvectors()
            # calc scalars    
            self.filestoclass[str(value)].calcscalars()
            # fill each vallist at the device index with the value of that scalar
            for scalarval in self.dummy.scalarlist:
                self.scalartoarrayofvalues[scalarval][z]=getattr(self.filestoclass[value], scalarval)
            # fill each timelist at the device index with the value of that timestamp
            for timeval in self.time_metrics:
                self.timetoarrayofvalues[timeval].append(getattr(self.filestoclass[value], timeval))
                            
# BUILD DATA DATAFRAME

        # setup lists
        self.data_vals = []
        self.data_rows = []
        self.data_cols = []             
        # fill scalar values, make it an array, transpose it
        for zz, scalar in enumerate(self.dummy.scalarlist):
            self.data_vals.append(self.scalartoarrayofvalues[scalar].tolist())
        self.data_array = np.asarray(self.data_vals).transpose()
        # fill file names (row names)
        for file in self.listoffiles:
            self.data_rows.append(file.rsplit(".", 1)[0])  
        # fill scalar names (column names)
        for scalar in self.dummy.scalarlist:
            self.data_cols.append(self.dummy.Label[scalar])  
        # create dataframe for all data
        self.allscalars = pd.DataFrame(data = self.data_array, 
                                       index = self.data_rows, 
                                       columns = self.data_cols)
                
# BUILD TIMEIINFO DATAFRAME

        # setup lists
        self.time_vals = []
        self.time_cols = []
        # fill timevalues and column labels from userinputs if its not time series
        if self.timeseries == 0:
            for zz, timeval in enumerate(self.time_metrics):
                self.time_vals.append(self.timetoarrayofvalues[timeval])
                self.time_cols.append(self.dummy.TimeLabels[timeval])
        # fill timevalues and column labels from userinputs if it is time series
        elif self.timeseries == 1:
            self.time_elapsed = []
            min_epoch = min(self.timetoarrayofvalues['epoch_time'])
            self.time_elapsed = self.timetoarrayofvalues['epoch_time'] 
            for z, value in enumerate(self.time_elapsed):
                self.time_elapsed[z] = value - min_epoch
            self.time_vals.append(self.time_elapsed)
            self.time_cols.append("Time Elapsed (sec)")
        # transpose matrix
        self.time_array = np.asarray(self.time_vals).transpose()
        # create dataframe for time metrics
        self.timeinfo = pd.DataFrame(data =self.time_array,
                                       index = self.data_rows,
                                       columns = self.time_cols)
                  
                
# COMBINE DATAFRAMES THAT HOLD DATA AND TIMESTAMPS, SORT BY TIME TAKEN, DISPLAY

        # combine dataframe for all data with time info
        self.valuesandtime = pd.concat([self.timeinfo,self.allscalars],axis=1,join='inner')    
        # if its a time series, sort it by time
        if self.timeseries == 1:
            self.valuesandtime = self.valuesandtime.sort_values("Time Elapsed (sec)")
        # display values and times
        display(self.valuesandtime)
        # if not a time series display stats as well
        if self.timeseries == 0:
            # create dataframe for average and stddev of data
            self.statistics = pd.DataFrame(data =[self.allscalars.mean(),self.allscalars.std()],
                                           index = ['Mean Value', 'Standard Deviation'],
                                           columns = self.data_cols)
            display(self.statistics)
                
        # note that this code makes numpy arrays for each scalar (e.g. self.jsc_list) as well as pandas data frame to hold all of the data
        # not sure what will be best moving forward for saving the files. 

       
                        

# Run Code

In [3]:
if __name__ == '__main__':
    XYTabulator()

FTO_C_round2_sample2_pixel3_fwd_light.csv is bad


,Date,Time,PCE (%),Jsc (mA/cm²),Voc (V),FF (%),Jmp (mA/cm²),Vmp (V),Pmp (mW/cm²),Rs (Ω/cm²),Rsh (Ω/cm²),Rch (Ω/cm²)
FTO_A_round1_sample1_pixel1_fwd_light,08/05/2021,05:16:34 PM,2.700103,7.288425,0.844272,43.879784,4.900187,0.551020,2.700103,0.027763,0.339270,0.120668
FTO_A_round1_sample1_pixel1_rev_light,08/05/2021,05:16:44 PM,3.031464,7.312018,0.846741,48.962622,5.286183,0.573469,3.031464,0.025500,0.689586,0.109369
FTO_A_round1_sample1_pixel2_fwd_light,08/04/2021,03:13:10 PM,1.451374,5.551178,0.658301,39.716400,3.486144,0.416327,1.451374,0.044307,0.253837,0.113862
FTO_A_round1_sample1_pixel2_rev_light,08/04/2021,03:12:50 PM,1.524761,5.608471,0.633679,42.903067,3.871156,0.393878,1.524761,0.035377,0.442220,0.109924
FTO_A_round1_sample1_pixel3_fwd_light,08/04/2021,03:22:06 PM,1.530001,4.790200,0.740996,43.104452,3.163293,0.483673,1.530001,0.035428,0.414559,0.138936
FTO_A_round1_sample1_pixel3_rev_light,08/04/2021,03:21:46 PM,1.639432,4.933486,0.721217,46.075905,3.554521,0.461224,1.639432,0.033439,0.992753,0.141174
FTO_A_round1_sample2_pixel1_fwd_light,08/04/2021,03:26:48 PM,0.336606,2.354480,0.649499,22.011438,1.030856,0.326531,0.336606,0.254241,0.143995,0.322505
FTO_A_round1_sample2_pixel1_rev_light,08/04/2021,03:26:28 PM,0.945702,3.001486,0.707618,44.526560,2.271540,0.416327,0.945702,0.117999,127.758420,0.193689
FTO_A_round1_sample2_pixel2_fwd_light,08/04/2021,03:28:56 PM,0.697005,3.970454,0.717432,24.468936,1.876553,0.371429,0.697005,0.129375,0.122373,0.202922
FTO_A_round1_sample2_pixel2_rev_light,08/04/2021,03:28:36 PM,1.745838,4.402352,0.787850,50.335654,3.449437,0.506122,1.745838,0.052677,7.123430,0.147528


,PCE (%),Jsc (mA/cm²),Voc (V),FF (%),Jmp (mA/cm²),Vmp (V),Pmp (mW/cm²),Rs (Ω/cm²),Rsh (Ω/cm²),Rch (Ω/cm²)
Mean Value,2.288851,6.183190,0.737743,48.735331,4.360162,0.514302,2.288851,0.033421,2.523779,0.131923
Standard Deviation,1.132778,2.043236,0.189326,10.775018,1.491909,0.156891,1.132778,0.028734,11.455368,0.059283
